In [ ]:
from pymongo import MongoClient
import pymongo
import pprint
import re
from IPython.display import clear_output

In [ ]:
from pymongo import MongoClient, InsertOne, UpdateOne
import pprint

from bson.json_util import loads

In [ ]:
!pip install dataparser

In [ ]:

client = MongoClient("mongodb+srv://senura96:@cluster0.qcqup.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")




In [ ]:
people_raw = client.cleansing['people-raw']

In [ ]:
batch_size = 1000
inserts = []
count = 0

In [ ]:
# There are over 50,000 lines, so this might take a while...
# Make sure to wait until the cell finishes executing before moving on (the * will turn into a number)
with open("./people-raw.json") as dataset: 
    for line in dataset: 
        inserts.append(InsertOne(loads(line)))
        
        count += 1
                       
        if count == batch_size:
            people_raw.bulk_write(inserts)
            inserts = []
            count = 0
if inserts:         
    people_raw.bulk_write(inserts)
    count = 0

In [ ]:
people_raw.count()

In [ ]:
people_with_string_birthdays = people_raw.find({"birthday" :{"$type":"string"}})

In [ ]:
people_with_string_birthdays.count()

In [ ]:
updates = []
# Again, we're updating several thousand documents, so this will take a little while
for person in people_with_string_birthdays:
    # Pymongo converts datetime objects into BSON Dates. The dateparser.parse function returns a
    # datetime object, so we can simply do the following to update the field properly.
    # Replace ZZZZ with the correct update operator
    updates.append(UpdateOne({ "_id": person["_id"] }, { "$set": { "birthday": dateparser.parse(person["birthday"]) } }))
    
    count += 1
                       
    if count == batch_size:
        people_raw.bulk_write(updates)
        updates = []
        count = 0
        
if updates:         
    people_raw.bulk_write(updates)
    count = 0

In [ ]:
people_with_string_birthdays.count()